## <a name="0">Amazon SageMaker에서 LLM 미세 조정</a>

이 솔루션에서는 생성형 인공지능(AI) 분야에서 강력한 기술인 사전 학습된 대규모 언어 모델(LLM)을 미세 조정하는 방법을 살펴봅니다. LLM은 방대한 양의 데이터로 사전 학습되어 언어의 미묘한 뉘앙스를 파악하고 일관성 있는 응답을 생성하는 데 매우 효과적입니다. 이러한 모델은 데이터에서 유용한 특징과 패턴을 추출하는 방법을 학습했기 때문에 다양한 머신러닝(ML) 작업에 매우 유용한 자원입니다.

전이 학습이라고도 하는 미세 조정을 통해 사전 학습된 모델이 얻은 지식을 활용하여 관련은 있지만 다른 작업에 적용할 수 있습니다. 모델을 처음부터 학습하는 대신 사전 학습된 모델에서 시작하여 특정 문제 영역에 맞게 수정합니다. 이 접근 방식은 상당한 컴퓨팅 리소스를 절약하고 사전 학습된 모델의 일반화 능력을 활용할 수 있다는 장점이 있습니다.

이 노트북은 사전 학습된 모델을 미세 조정하는 단계별 과정을 안내합니다. 주요 단계는 다음과 같습니다.

1. <a href="#step1">GPU 메모리 확인</a>
2. <a href="#step2">라이브러리 가져오기</a>
3. <a href="#step3">학습 데이터셋 준비</a>
4. <a href="#step4">사전 학습된 LLM 불러오기</a>
5. <a href="#step5">트레이너 정의 및 LLM 미세 조정</a>
6. <a href="#step6">미세 조정된 모델 배포</a>
7. <a href="#step7">배포된 추론 테스트</a>

참고: 이 노트북의 위쪽부터 아래쪽으로 순서대로 진행하고, 각 섹션을 건너뛰지 마십시오. 그렇지 않으면 코드 누락으로 인한 오류 메시지가 발생할 수 있습니다.

## <a name="step1">Step 1: Check GPU memory</a>

To check the GPU memory, run the following command.

In [ ]:
!nvidia-smi

If your CUDA memory is occupied by more than half (as in the following image), you need to shut down other running notebooks.

<p style="padding: 10px; border: 1px solid black;">
<img src="images/memory.png" alt="drawing" width="800"/> <br/>

## <a name="step2">2단계: 라이브러리 가져오기</a>

Hugging Face Transformers 라이브러리와 Transformers의 종속성 라이브러리인 PyTorch 라이브러리를 포함하여 필요한 라이브러리를 가져오려면 다음 두 코드 블록을 순서대로 하나씩 실행하세요.

In [ ]:
%%capture
pip install -r requirements.txt

In [ ]:
%%capture

import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings

import os
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Tuple, Union
from datasets import Dataset, load_dataset, disable_caching
disable_caching() ## disable huggingface cache

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import TextDataset

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer
import accelerate
import bitsandbytes

from IPython.display import Markdown

## <a name="step3">3단계: 학습 데이터셋 준비</a>

데이터셋을 로드하고 확인합니다. 이 실습에서는 두 개의 열(`instruction` 및 `response`)로 구성된 [Amazon SageMaker FAQs](https://aws.amazon.com/sagemaker/faqs/)를 기본 데이터셋으로 사용합니다.

In [ ]:
sagemaker_faqs_dataset = load_dataset("csv",
                                      data_files='data/amazon_sagemaker_faqs.csv')['train']
sagemaker_faqs_dataset

In [ ]:
sagemaker_faqs_dataset[0]

### <a name="step3">3.1단계: 프롬프트 준비</a>
LLM을 미세 조정하려면 아래와 같이 명령어 데이터셋에 PROMPT를 추가해야 합니다.

In [ ]:
from utils.helpers import INTRO_BLURB, INSTRUCTION_KEY, RESPONSE_KEY, END_KEY, RESPONSE_KEY_NL, DEFAULT_SEED, PROMPT
'''
PROMPT = """{intro}
            {instruction_key}
            {instruction}
            {response_key}
            {response}
            {end_key}"""
'''
Markdown(PROMPT)

이제 `_add_text` 파이썬 함수를 통해 프롬프트를 데이터셋에 추가합니다. 이 함수는 레코드를 입력으로 받으며, 두 필드(지시/응답)가 모두 null이 아닌지 확인한 후, 해당 값을 미리 정의된 프롬프트 템플릿에 전달합니다.

In [ ]:
def _add_text(rec):
    instruction = rec["instruction"]
    response = rec["response"]

    if not instruction:
        raise ValueError(f"Expected an instruction in: {rec}")

    if not response:
        raise ValueError(f"Expected a response in: {rec}")

    rec["text"] = PROMPT.format(
        instruction=instruction, response=response)

    return rec

In [ ]:
sagemaker_faqs_dataset = sagemaker_faqs_dataset.map(_add_text)
sagemaker_faqs_dataset[0]

Use `Markdown` to neatly display the text.

In [ ]:
Markdown(sagemaker_faqs_dataset[0]['text'])

## <a name="#step4">4단계: 사전 학습된 LLM 로드</a>

사전 학습된 모델을 로드하려면 Hugging Face Transformers 라이브러리의 `databricks/dolly-v2-3b` 모델을 사용하여 토크나이저와 기본 모델을 초기화합니다. 토크나이저는 원시 텍스트를 토큰으로 변환하고, 기본 모델은 주어진 프롬프트에 따라 텍스트를 생성합니다. 앞서 설명한 지침을 따르면 이러한 구성 요소를 올바르게 인스턴스화하고 코드에서 해당 기능을 사용할 수 있습니다.

토크나이저를 인스턴스화하려면 `AutoTokenizer.from_pretrained()` Python 함수를 사용합니다.

- `padding_side="left"`는 시퀀스에 패딩 토큰이 추가되는 위치를 지정합니다. 이 경우 패딩 토큰은 각 시퀀스의 왼쪽에 추가됩니다.

- `eos_token`은 시퀀스의 끝을 나타내는 특수 토큰입니다. 토큰을 `pad_token`에 할당하면 토큰화 중에 추가되는 모든 패딩 토큰도 시퀀스 끝 토큰으로 간주됩니다. 이는 모델을 통해 텍스트를 생성할 때 유용할 수 있는데, 모델은 패딩 토큰을 만나면 텍스트 생성을 중지해야 할 시점을 알 수 있기 때문입니다.
- `tokenizer.add_special_tokens...` 함수는 토크나이저의 어휘에 세 가지 특수 토큰을 추가합니다. 이러한 토큰은 토크나이저를 사용하는 애플리케이션에서 특정 목적을 수행할 가능성이 높습니다. 예를 들어, 이러한 토큰은 대화 시스템에서 입력, 명령 또는 응답의 끝을 표시하는 데 사용될 수 있습니다.

이 함수가 실행되면 `tokenizer` 객체가 초기화되어 텍스트 토큰화에 사용할 준비가 됩니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("verseAI/databricks-dolly-v2-3b",
                                          padding_side="left")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens":
                              [END_KEY, INSTRUCTION_KEY, RESPONSE_KEY_NL]})

- AutoTokenizer.from_pretrained(...)
  - 사전 학습된 모델(databricks-dolly-v2-3b)에 맞는 토크나이저를 자동으로 로드합니다.
  - 모델과 토크나이저는 항상 짝이 맞아야 합니다. (토큰 분해 방식이 다르면 성능/추론 오류 발생)

- padding_side="left"
  - 시퀀스 길이를 맞추기 위해 패딩(pad)을 왼쪽에 추가합니다.
  - 이는 Decoder-only 계열 LLM (GPT 계열) 에서 매우 중요합니다.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "verseAI/databricks-dolly-v2-3b",
    # use_cache=False,
    device_map="auto", #"balanced",
    load_in_8bit=True,
)

### <a name="#step4.1">4.1단계: 모델 학습 준비</a>
매개변수 효율 미세 조정(PEFT)을 사용하여 INT8 모델을 학습하기 전에 몇 가지 전처리가 필요합니다. 따라서 다음과 같은 작업을 수행하는 유틸리티 함수 `prepare_model_for_int8_training`을 가져옵니다.

- 안정성을 위해 모든 비 INT8 모듈을 전체 정밀도(FP32)로 캐스팅합니다.

- 입력 은닉 상태의 기울기 계산을 활성화하기 위해 입력 임베딩 레이어에 forward_hook을 추가합니다.

- 메모리 효율적인 학습을 위해 기울기 체크포인팅을 활성화합니다.

In [ ]:
model.resize_token_embeddings(len(tokenizer))

`preprocess_batch` 함수를 사용하여 배치 데이터의 텍스트 필드를 전처리합니다. 이 함수는 지정된 최대 길이를 기준으로 토큰화, 잘림 및 기타 관련 연산을 적용합니다. 이 함수는 데이터 배치, 토크나이저 및 최대 길이를 입력으로 받습니다.

자세한 내용은 `mlu_utils/helpers.py` 파일을 참조하십시오.

In [ ]:
from functools import partial
from utils.helpers import mlu_preprocess_batch

MAX_LENGTH = 256
_preprocessing_function = partial(mlu_preprocess_batch, max_length=MAX_LENGTH, tokenizer=tokenizer)

다음으로, 데이터셋의 각 배치에 전처리 함수를 적용하여 텍스트 필드를 적절히 수정합니다. 매핑 작업은 배치 방식으로 수행되며, 지시사항, 응답 및 텍스트 열이 데이터셋에서 제거됩니다. 마지막으로, `sagemaker_faqs_dataset`에서 `input_ids` 필드의 길이를 기준으로 필터링하여 지정된 `MAX_LENGTH` 내에 맞도록 하여 `processed_dataset`을 생성합니다.

In [ ]:
encoded_sagemaker_faqs_dataset = sagemaker_faqs_dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "response", "text"],
)

processed_dataset = encoded_sagemaker_faqs_dataset.filter(lambda rec: len(rec["input_ids"]) < MAX_LENGTH)

Split the dataset into `train` and `test` for evaluation.

In [ ]:
split_dataset = processed_dataset.train_test_split(test_size=14, seed=0)
split_dataset

## <a name="#step5">5단계: 트레이너 정의 및 LLM 미세 조정</a>

이 실습에서는 효율적인 모델 미세 조정을 위해 [LoRA: Low-Rank Adaptation](https://arxiv.org/abs/2106.09685)을 사용합니다. LoRA는 사전 학습된 모델의 가중치를 고정하고 학습 가능한 랭크 분해 행렬을 Transformer 아키텍처의 각 레이어에 주입하여 하위 작업에 필요한 학습 가능한 파라미터 수를 크게 줄입니다. Adam으로 미세 조정된 GPT-3 175B와 비교했을 때, LoRA는 학습 가능한 파라미터 수를 10,000배 줄이고 GPU 메모리 요구량을 3배까지 줄일 수 있습니다.

### <a name="#step5.1">5.1단계: LoraConfig 정의 및 LoRA 모델 로드</a>

[huggingface 🤗 PEFT: 최첨단 파라미터 효율 미세 조정](https://github.com/huggingface/peft)의 빌드된 LoRA 클래스 `LoraConfig`를 사용합니다. `LoraConfig` 내에서 다음 매개변수를 지정합니다.

- `r`: 저차 행렬의 차원
- `lora_alpha`: 저차 행렬의 스케일링 계수
- `lora_dropout`: LoRA 레이어의 드롭아웃 확률

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# First prepare the model for int8 training
model = prepare_model_for_int8_training(model)

# Then freeze all parameters
for param in model.parameters():
    if param.dtype == torch.float32 or param.dtype == torch.float16:
        param.requires_grad = False

# Unfreeze only the top N layers
num_layers_to_unfreeze = 2  # Adjust this number as needed
for i, layer in enumerate(model.gpt_neox.layers[-num_layers_to_unfreeze:]):
    for param in layer.parameters():
        if param.dtype == torch.float32 or param.dtype == torch.float16:
            param.requires_grad = True
        
MICRO_BATCH_SIZE = 8
BATCH_SIZE = 64
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LORA_R = 256 # 512
LORA_ALPHA = 512 # 1024
LORA_DROPOUT = 0.05

        
# Define LoRA Config
lora_config = LoraConfig(
                 r=LORA_R,
                 lora_alpha=LORA_ALPHA,
                 lora_dropout=LORA_DROPOUT,
                 bias="none",
                 task_type="CAUSAL_LM"
)

Use the `get_peft_model` function to initialize the model with the LoRA framework, configuring it based on the provided `lora_config` settings. This way, the model can incorporate the benefits and capabilities of the LoRA optimization approach.

“3B급 LLM을 int8로 로드한 뒤,
전체 파라미터는 고정하고
상위 레이어 + LoRA 어댑터만 학습해서
적은 GPU로 안정적인 Instruction Fine-tuning을 하겠다.”

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

보시는 바와 같이, LoRA 전용 학습 가능 파라미터는 전체 가중치의 약 3%에 불과하여 훨씬 효율적입니다.

### <a name="#step5.2">5.2단계: 데이터 콜레이터 정의</a>

DataCollator는 데이터셋에서 샘플 목록을 받아 PyTorch 텐서 딕셔너리 형태로 배치로 콜레이션하는 transformers 함수입니다.

Transformers 라이브러리의 기본 `DataCollatorForLanguageModeling` 클래스의 기능을 확장한 `DataCollatorForCompletionOnlyLM`을 사용하세요. 이 사용자 지정 콜레이터는 입력 텍스트에 프롬프트 다음에 응답이 오는 경우와 그에 따라 레이블이 수정되는 경우를 처리하도록 설계되었습니다.

구현은 `utils/helpers.py`를 참조하세요.

In [ ]:
from utils.helpers import MLUDataCollatorForCompletionOnlyLM

data_collator = MLUDataCollatorForCompletionOnlyLM(
        tokenizer=tokenizer, mlm=False, return_tensors="pt", pad_to_multiple_of=8
)

### <a name="#step5.3">5.3단계: 트레이너 정의</a>

LLM을 미세 조정하려면 트레이너를 정의해야 합니다. 먼저 훈련 인수를 정의합니다.

In [ ]:
EPOCHS = 10
LEARNING_RATE = 1e-4
MODEL_SAVE_FOLDER_NAME = "dolly-3b-lora"

training_args = TrainingArguments(
                    output_dir=MODEL_SAVE_FOLDER_NAME,
                    fp16=True,
                    per_device_train_batch_size=8,
                    per_device_eval_batch_size=8,
                    learning_rate=LEARNING_RATE,
                    num_train_epochs=EPOCHS,
                    logging_strategy="steps",
                    logging_steps=100,
                    evaluation_strategy="steps",
                    eval_steps=100,
                    save_strategy="steps",
                    save_steps=20000,
                    save_total_limit=10,
                    optim="adamw_torch"
)

This is where the magic happens! Initialize the trainer with the defined model, tokenizer, training arguments, data collator, and the train/eval datasets.

The training takes about 10 minutes.

In [ ]:
trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=split_dataset['train'],
        eval_dataset=split_dataset["test"],
        data_collator=data_collator,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

### <a name="#step5.4">5.4단계: 미세 조정된 모델 저장</a>

학습이 완료되면 `transformers.PreTrainedModel.save_pretrained` 함수를 사용하여 모델을 디렉토리에 저장할 수 있습니다.

이 함수는 학습된 증분 PEFT 가중치(adapter_model.bin)만 저장하므로 모델을 저장, 전송 및 불러오는 데 매우 효율적입니다.

In [ ]:
trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)

If you want to save the full model that you just fine-tuned, you can use the [`transformers.trainer.save_model`] function. Meanwhile, save the training arguments together with the trained model.

In [ ]:
trainer.save_model()

In [ ]:
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

Save the tokenizer along with the trained model.

In [ ]:
tokenizer.save_pretrained(MODEL_SAVE_FOLDER_NAME)

## <a name="#step6">6단계: 미세 조정된 모델 배포</a>

### <a name="step6">배포 매개변수 개요</a>

Amazon SageMaker Python SDK와 Deep Java Library(DJL)를 사용하여 배포하려면 다음 매개변수를 사용하여 `Model` 클래스를 인스턴스화해야 합니다.
```{python}
model = Model(
image_uri,
model_data=...,
predictor_cls=...,
role=aws_role
)
```
- `image_uri`: 사용할 딥러닝 프레임워크 및 버전을 나타내는 Docker 이미지 URI입니다.

- `model_data`: Amazon Simple Storage Service(Amazon S3) 버킷에 있는 미세 조정된 LLM 모델 아티팩트의 위치입니다. 모델의 매개변수, 아키텍처 및 필요한 아티팩트가 포함된 TAR GZ 파일의 경로를 지정합니다.

- `predictor_cls`: DJL과 관련이 없는 JSON 입력 JSON 출력 예측기입니다. 자세한 내용은 [sagemaker.djl_inference.DJLPredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/djl/sagemaker.djl_inference.html#djlpredictor)를 참조하세요.

- `role`: 모델 데이터가 포함된 S3 버킷과 같은 리소스에 액세스하는 데 필요한 권한을 제공하는 AWS Identity and Access Management(IAM) 역할 ARN입니다.

### <a name="step6.1">6.1단계: SageMaker 매개변수 인스턴스화</a>

Amazon SageMaker 세션을 초기화하고 SageMaker 역할 및 AWS 리전과 같은 AWS 환경 관련 정보를 가져옵니다. 또한 SageMaker 세션의 리전을 사용하여 "djl-deepspeed" 프레임워크의 특정 버전에 대한 이미지 URI를 지정합니다. 이미지 URI는 Amazon SageMaker 또는 Amazon Elastic Container Registry(Amazon ECR)와 같은 다양한 AWS 서비스에서 사용할 수 있는 특정 Docker 컨테이너 이미지에 대한 고유 식별자입니다.

In [ ]:
# installing sagemaker library
!pip3 install sagemaker==2.237.1

In [ ]:
import boto3
import json
import sagemaker.djl_inference
from sagemaker.session import Session
from sagemaker import image_uris
from sagemaker import Model

sagemaker_session = Session()
print("sagemaker_session: ", sagemaker_session)

aws_role = sagemaker_session.get_caller_identity_arn()
print("aws_role: ", aws_role)

aws_region = boto3.Session().region_name
print("aws_region: ", aws_region)

image_uri = image_uris.retrieve(framework="djl-deepspeed",
                                version="0.22.1",
                                region=sagemaker_session._region_name)
print("image_uri: ", image_uri)

### <a name="step6.2">6.2단계: 모델 아티팩트 생성</a> ###

모델 아티팩트를 S3 버킷에 업로드하려면 모델 매개변수가 포함된 TAR GZ 파일을 생성해야 합니다. 먼저 `lora_model`이라는 디렉토리와 `dolly-3b-lora`라는 하위 디렉토리를 생성합니다. "-p" 옵션을 사용하면 중간 디렉토리가 없는 경우 명령이 자동으로 생성합니다. 그런 다음 LoRA 체크포인트 파일인 `adapter_model.bin`과 `adapter_config.json`을 `dolly-3b-lora` 디렉토리에 복사합니다. 기본 Dolly 모델은 런타임 시 Hugging Face Hub에서 다운로드됩니다.

In [ ]:
%%bash
rm -rf lora_model
mkdir -p lora_model
mkdir -p lora_model/dolly-3b-lora
cp dolly-3b-lora/adapter_config.json lora_model/dolly-3b-lora/
cp dolly-3b-lora/adapter_model.bin lora_model/dolly-3b-lora/

다음으로, `serving.properties` 파일에 [DJL Serving 구성 옵션](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html)을 설정합니다. Jupyter의 `%%writefile` 매직 명령어를 사용하여 다음 내용을 `lora_model/serving.properties`라는 파일에 기록할 수 있습니다.

- `engine=Python`: 이 줄은 서빙에 사용할 엔진을 지정합니다.

- `option.entryPoint=model.py`: 이 줄은 서빙 프로세스의 진입점을 지정하며, `model.py`로 설정됩니다.

- `option.adapter_checkpoint=dolly-3b-lora`: 이 줄은 어댑터의 체크포인트를 `dolly-3b-lora`로 설정합니다. 체크포인트는 일반적으로 모델 또는 해당 매개변수의 저장된 상태를 나타냅니다.

- `option.adapter_name=dolly-lora`: 이 줄은 어댑터의 이름을 dolly-lora로 설정합니다. dolly-lora는 모델과 서비스 인프라 간의 인터페이스를 지원하는 구성 요소입니다.

In [ ]:
%%writefile lora_model/serving.properties
engine=Python
option.entryPoint=model.py
option.adapter_checkpoint=dolly-3b-lora
option.adapter_name=dolly-lora

You also need the environment requirement file in the model artifact. Create a file named `lora_model/requirements.txt` and write a list of Python package requirements, typically used with package managers such as `pip`.

In [ ]:
%%writefile lora_model/requirements.txt
accelerate>=0.16.0,<1
bitsandbytes==0.39.0
click>=8.0.4,<9
datasets>=2.10.0,<3
deepspeed>=0.8.3,<0.9
faiss-cpu==1.7.4
ipykernel==6.22.0
scipy==1.11.1
torch>=2.0.0
transformers==4.28.1
peft==0.3.0
pytest==7.3.2

### <a name="step6.3">6.3단계: 추론 스크립트 생성</a>

미세 조정 노트북과 마찬가지로, `InstructionTextGenerationPipeline`이라는 사용자 지정 파이프라인이 정의됩니다. 코드는 `utils/deployment_model.py`에 제공됩니다.

이러한 추론 함수들을 `lora_model/model.py`에 저장합니다.

In [ ]:
%%bash
cp utils/deployment_model.py lora_model/model.py

### <a name="step6.4">6.4단계: 모델 아티팩트를 Amazon S3에 업로드</a>

lora_model 디렉토리를 압축된 tarball 아카이브로 생성하고 lora_model.tar.gz로 저장합니다.

In [ ]:
%%bash
tar -cvzf lora_model.tar.gz lora_model/

Upload the lora_model.tar.gz file to the specified S3 bucket.

In [ ]:
import boto3
import json
import sagemaker.djl_inference
from sagemaker.session import Session
from sagemaker import image_uris
from sagemaker import Model

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

s3 = boto3.resource('s3')

# Get the name of the bucket with prefix lab-code
for bucket in s3.buckets.all():
    if bucket.name.startswith('artifact'):
        mybucket = bucket.name
        print(mybucket)

response = s3_client.upload_file("lora_model.tar.gz", mybucket, "lora_model.tar.gz")

### <a name="step6.5">6.5단계: 모델 배포</a> ###

이제 SageMaker Python SDK를 사용하여 미세 조정된 LLM 모델을 배포할 차례입니다. SageMaker Python SDK의 `Model` 클래스는 다음 매개변수를 사용하여 인스턴스화됩니다.

- `image_uri`: 사용할 딥러닝 프레임워크 및 버전을 나타내는 Docker 이미지 URI입니다.

- `model_data`: S3 버킷에 있는 미세 조정된 LLM 모델 아티팩트의 위치입니다. 모델의 매개변수, 아키텍처 및 필요한 아티팩트가 포함된 TAR GZ 파일의 경로를 지정합니다.

- `predictor_cls`: 이는 JSON 입력, JSON 출력 예측기이며 DJL과는 관련이 없습니다. 자세한 내용은 [sagemaker.djl_inference.DJLPredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/djl/sagemaker.djl_inference.html#djlpredictor)를 참조하세요.
- `role`: 모델 데이터가 포함된 S3 버킷과 같은 리소스에 액세스하는 데 필요한 권한을 제공하는 IAM 역할 ARN입니다.

In [ ]:
model_data="s3://{}/lora_model.tar.gz".format(mybucket)

model = Model(image_uri=image_uri,
              model_data=model_data,
              predictor_cls=sagemaker.djl_inference.DJLPredictor,
              role=aws_role)

NOTE: The deployment should be completed within 10 minutes. Any longer than that, your endpoint might have failed.

In [ ]:
%%time
predictor = model.deploy(1, "ml.g4dn.2xlarge")

### <a name="step7">Step 7: Test the deployed inference</a>

Test the inference endpoint with [predictor.predict](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor.predict).

In [ ]:
outputs = predictor.predict({"inputs": "What security measures does Amazon SageMaker have?"})

In [ ]:
from IPython.display import Markdown
Markdown(outputs)